In [9]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
import os 
from time import time
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.datasets import cifar10
import random
import cv2
import numpy as np
from keras.utils import to_categorical

# Basic Guides:
# https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html

batch_size = 128
Classes = 12
N_training = 4000
N_verification = 495

In [2]:
def load_data_plants(source):
    import shutil
    
    source1 = "C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_180/train"
    dest11 =  "C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_180/verf"
    folders = os.listdir(source)

    x_data = []
    
    for f in folders:
        files = os.listdir(source + '/'+ f)
        for i in files:
            x = image.load_img(source + '/'+ f + '/' + i)
            x = img_to_array(x)
            x_data.append(x)
    return x_data

In [3]:
# Configure the loading of data
# Image Generator
# Guied: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

# Load data for data augmentation
x_train = load_data_plants("C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_180/train")
x_verf = load_data_plants("C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_180/verf")

# fit the data augmentation
train_datagen.fit(x_train)

# fit the data augmentation
val_datagen.fit(x_verf)
# Clear to save mem
x_train=[]
x_verf=[]
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../../LocalContent/Datasets/Resized_180/train',  # this is the target directory
       # target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = val_datagen.flow_from_directory(
        '../../LocalContent/Datasets/Resized_180/verf',
      #  target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 4255 images belonging to 12 classes.
Found 495 images belonging to 12 classes.


In [4]:
# Configure model
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(Classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [5]:
# Fit the model for the top layers
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# https://stackoverflow.com/questions/45943675/meaning-of-validation-steps-in-keras-sequential-fit-generator-parameter-list
model.fit_generator(
        train_generator,
        steps_per_epoch=np.floor(N_training/batch_size),
        epochs=1,
        validation_data=validation_generator,
        validation_steps=(N_verification/batch_size),
        max_queue_size=1)

Epoch 1/1
62/62 [==============================] - 134s 2s/step - loss: 2.9981 - val_loss: 2.6967


In [11]:
# Fit the model for the lower layers
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

tbCallBack = keras.callbacks.TensorBoard(log_dir="../../LocalContent/Logs/08_KerasInceptionV3_TestingDirectionLoading/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers

history = model.fit_generator(
        train_generator,
        steps_per_epoch=np.floor(N_training/batch_size),
        epochs=5,
        validation_data=validation_generator,
        validation_steps=np.floor(N_verification/batch_size),
        max_queue_size=1,
        callbacks=[tbCallBack])


Epoch 1/5
31/31 [==============================] - 75s 2s/step - loss: 0.8145 - acc: 0.7908 - val_loss: 1.9194 - val_acc: 0.3698
Epoch 2/5
31/31 [==============================] - 68s 2s/step - loss: 0.7831 - acc: 0.8016 - val_loss: 1.8940 - val_acc: 0.3750
Epoch 3/5
 2/31 [>.............................] - ETA: 58s - loss: 0.8198 - acc: 0.7969 

KeyboardInterrupt: 